In [2]:
import nltk
from nltk.corpus import wordnet as wn
import requests
import pandas as pd

In [3]:

nltk.download('wordnet')
nltk.download('omw-1.4')

def get_synonyms(word):
    synonyms = set()
    for syn in wn.synsets(word):
        for lemma in syn.lemmas():
            synonyms.add(lemma.name().replace('_', ' '))
    return synonyms

# Example synonyms for 'quiet'
quiet_synonyms = get_synonyms('quiet')
print(quiet_synonyms)


[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/hanzhitan/nltk_data...
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/hanzhitan/nltk_data...


{'hush', 'restrained', 'tranquillity', 'smooth', 'quiet down', 'muted', 'quiet', 'hushed', 'tranquillize', 'quieten', 'tranquil', 'tranquility', 'silence', 'placid', 'calm down', 'tranquilize', 'pipe down', 'repose', 'serenity', 'unruffled', 'quiesce', 'still', 'lull', 'subdued', 'placidity', 'calm', 'tranquillise', 'quietly'}


In [ ]:
def fetch_places_nearby(api_key, location, radius, place_type):
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {
        'key': api_key,
        'location': location,  # "latitude,longitude"
        'radius': radius,
        'type': place_type
    }
    response = requests.get(url, params=params)
    results = response.json().get('results', [])
    places = []
    for place in results:
        places.append({
            'name': place.get('name'),
            'address': place.get('vicinity'),
            'type': place_type,
            'rating': place.get('rating', None),
            'user_ratings_total': place.get('user_ratings_total', 0),
            'latitude': place['geometry']['location']['lat'],
            'longitude': place['geometry']['location']['lng'],
        })
    return places

# Example usage
api_key = 'YOUR_API_KEY_HERE'
places = []
for place_type in ['library', 'cafe', 'bar']:  # Add more types if needed
    places.extend(fetch_places_nearby(api_key, '41.8781,-87.6298', 1000, place_type))


In [ ]:
labels_with_synonyms = {
    'quiet': get_synonyms('quiet'),
    'group_work': get_synonyms('group_work'),
    'wifi_available': ['wifi', 'free wifi', 'wireless internet', 'internet available'],
    'ambience': {get_synonyms('modern'), get_synonyms('bustling'), get_synonyms('cozy')},
}

def label_extraction(place_info):
    description = (place_info.get('name', '') + ' ' + place_info.get('address', '')).lower()
    features = {label: 0 for label in labels_with_synonyms.keys()}
    
    for label, synonyms in labels_with_synonyms.items():
        if label == 'ambience':
            # Ambience requires checking for multiple types
            features[label] = []
            for ambiance_type in synonyms:
                if any(word in description for word in get_synonyms(ambiance_type)):
                    features[label].append(ambiance_type)
        else:
            # Other features are binary
            if any(word in description for word in synonyms):
                features[label] = 1
    
    return features

# Example place for testing
place_info = {
    'name': 'Java City',
    'address': '123 Brew St, Free WiFi available, cozy seating ideal for group work'
}

# Test the function
print(label_extraction(place_info))



# {
#     "quiet": 0,
#     "wifi_available": 1,
#     "ambience": ["cozy"],
#     "group_work": 1
# }